### 개체명 인식: NER
- NER(Named Entity Recognition, 개체명 인식)에서는 문장 속에서 사람, 장소, 조직, 날짜 같은 의미 있는 단어들을 찾아내는 작업을 함.
- 텍스트에서 특정 의미를 가진 단어나 구절을 찾아내고 분류하는 작업
- 이때 단어(token)가 개체인지 표시하는 형식이 BIO 방식

In [ ]:
# 홍길동은 2025년 11월 19일 서울시청에서 삼성전자 직원을 만났다
# 홍길동 - [인명]
# 2024년 1월 15일 - [날짜]
# 서울시청 - [지명]
# 삼성전자 - [기관명]

# 활용분야
    # 뉴스기사 : 기사에서 인물, 장소, 기관 자동 추출
    # 의료문서 : 병명, 약물명, 증상
    # 계약서 : 회사명, 날짜, 금액
    # 챗봇 : 사용자 질문에 핵심정보 파악

# BIO 태깅
    # B(Begin) 개체 시작
    # I(Inside) 개체 내부
    # O(Outside) 개체가 아님

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel

In [ ]:
# BIO 태깅
tokens = ["김철수는", "2024년", "1월", "15일", "서울시청에서", "삼성전자", "직원을", "만났다"]
bio_tags = ["B-PER", "B-DAT", "I-DAT", "I-DAT", "B-LOC", "B-ORG", "O", "O"]
for token, tag in zip(tokens, bio_tags):
    if tag.startswith('B-'):
        desc = f"'{tag[2:]}' 개체의 시작"
    elif tag.startswith('I-'):
        desc = f"'{tag[2:]}' 개체의 내부"
    else :
        desc = "개체가 아님"
    print(f" {token:12} | {tag:8} | {desc}")

 김철수는         | B-PER    | 'PER' 개체의 시작
 2024년        | B-DAT    | 'DAT' 개체의 시작
 1월           | I-DAT    | 'DAT' 개체의 내부
 15일          | I-DAT    | 'DAT' 개체의 내부
 서울시청에서       | B-LOC    | 'LOC' 개체의 시작
 삼성전자         | B-ORG    | 'ORG' 개체의 시작
 직원을          | O        | 개체가 아님
 만났다          | O        | 개체가 아님


In [ ]:
# 학습데이터
train_sentences = [
    ["김철수는", "서울에", "산다"],
    ["이영희는", "2024년에", "부산으로", "이사했다"],
    ["삼성전자는", "대한민국의", "대기업이다"],
    ["박지성은", "축구선수다"],
    ["2025년", "1월", "1일은", "새해다"],
]

train_labels = [
    ["B-PER", "B-LOC", "O"],
    ["B-PER", "B-DAT", "B-LOC", "O"],
    ["B-ORG", "B-LOC", "O"],
    ["B-PER", "O"],
    ["B-DAT", "I-DAT", "I-DAT", "O"],
]

In [ ]:
from ast import mod
# 토크나이저
MODEL_NAME = 'skt/kobert-base-v1'
tokenizer=AutoTokenizer.from_pretrained(MODEL_NAME)
text = '김철수는 서울에 산다'
# 토크나이저
tokens = tokenizer.tokenize(text)
# 인코딩
encoded = tokenizer(text, return_tensors='pt')
encoded

{'input_ids': tensor([[517, 490, 494,   0, 517,   0, 491,   0, 491,   0, 517,   0,   0,   0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
# NER 모델 4단계로 구성
# 1. 입력 테스트
# 2. koBERT 인코더 : 문장의 의미를 이해
# 3. 분류기(Linear) : 예측
# 4. 출력 라벨 : B-PER B-LOC

In [ ]:
import torch.nn as nn
import numpy as np
class SimpleNERModel(nn.Module):
  def __init__(self, num_labels) -> None:
    super(SimpleNERModel, self).__init__()
    self.num_labels = num_labels
    self.bert = AutoModel.from_pretrained(MODEL_NAME)
    self.dropout = nn.Dropout(0.1)
    self.clf = nn.Linear(self.bert.config.hidden_size,  self.num_labels) #분류기
  def forward(self, input_ids, attention_maks):
    # kobert로 문자 인코딩
    outputs = self.bert(input_ids, attention_mask=attention_maks)
    # 마지막 은닉상태 추출
    sequence_output =  outputs.last_hidden_state
    # Dropout 적용
    sequence_output = self.dropout(sequence_output)
    # 분류기
    logits = self.clf(sequence_output)
    return logits
# 라벨은 모델 라벨의 갯수 ??
label_list = set([data for i in train_labels for data in i])
label_list
model = SimpleNERModel(num_labels=len(label_list))
